In [1]:
DEARGADAOL_LANDING = "https://www.leighleat.com/sc%C3%A9alta/45"

In [2]:
import requests
from bs4 import BeautifulSoup

In [14]:
def get_page_list(url: str):
    PAGE_REQ = requests.get(url)
    if PAGE_REQ.status_code != 200:
        return None

    soup = BeautifulSoup(PAGE_REQ.content, "html.parser")

    pages = set()
    for row in soup.find_all("div", {'class': 'row'}):
        for anchor in row.find_all("a"):
            if anchor["href"].startswith("/pages/"):
                pages.add(f'https://www.leighleat.com{anchor["href"]}')
    return list(pages)

In [55]:
def get_page(url: str):
    PAGE_REQ = requests.get(url)
    if PAGE_REQ.status_code != 200:
        return None

    soup = BeautifulSoup(PAGE_REQ.content, "html.parser")

    audio_url = ""
    audio = soup.find("audio", {"id": "audio"})
    audio_source = audio.find("source")
    if audio_source:
        if audio_source["src"].startswith("http"):
            audio_url = audio_source["src"]
        elif audio_source["src"].startswith("/"):
            audio_url = f'https://www.leighleat.com{audio_source["src"]}'

    text_pieces = []
    content = soup.find("div", {"class": "story-page-content"})
    if not content:
        content = soup.find("div", {"class": "inner-container"})
        if content:
            for child in content.children:
                print(child)
                if child.name == "h1" and "page-text" in child["class"]:
                    if child.text.strip() != "":
                        text_pieces.append(child.text.strip())
                elif child.name == "div" and child["class"] == "text":
                    print("div", child)
                    for subchild in child.children:
                        if subchild.name == "p":
                            print("para", subchild.text)
                            if subchild.text.strip() != "":
                                text_pieces.append(subchild.text.strip())
    if len(text_pieces) != 0:
        text = " ".join(text_pieces)
    else:
        text = content.text.strip()
    return audio_url, text
    

In [41]:
get_page("https://www.leighleat.com/pages/475")

('https://www.leighleat.com/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBcWNEIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--875ea38698f12dbef3db303886c58fd6d5190cab/lth%201.mp3',
 'Fear siúil as Dúiche Sheoighe a tháinig isteach i dteach m’athar a d’inis an scéal seo dúinn \ncois teallaigh oíche gharbh gheimhridh. Bhí an ghaoth ag caoineadh thart timpeall an tí, \nar nós mná ag caoineadh marbh, le linn labhartha dó, agus níodh sé a ghlór a ardú nó a ísliú \nde réir mar a d’ardaíodh nó mar a d’íslíodh glór na gaoithe. Fear ard a bhí ann, súile fiáine aige, agus a chuid éadaigh beagnach ina mbalcaisí. Bhí saghas eagla orm roimhe nuair a tháinig sé isteach, agus níor lúide m’eagla a scéal.\n‘Na trí feithidí is beannaithe ar an domhan,’ arsa an fear siúil, ‘an chadóg, an spideoigín agus bó Dé. Agus na trí feithidí is mallaithe ar an domhan, an nathair nimhe, an dreoilín agus an deargadaol.\nAgus ’sé an deargadaol an fheithide is mallaithe acu. Is agamsa atá a fhios si

In [56]:
get_page("https://www.leighleat.com/pages/2177")



<h1 class="page-title">Caibidil 1: An Rí Artúr </h1>


<div class="text">
<p>
Fadó fadó bhí laoch óg misniúil ann darbh ainm Artúr. Theastaigh uaidh a thír a thabhairt slán ó ionradh agus ó ansmacht. Chabhraigh draoi darbh ainm Merlin leis. Rinne Merlin claíomh draíochta a bhí greamaithe go docht i gcarraig mhór. Seo mar a bhí scríofa ar an gcarraig: Pé duine atá ábalta an claíomh seo a tharraingt as an gcarraig, beidh an duine sin ina cheannaire dlistineach ar an tír. 
<br/>Rinne na fir ba láidre sa tír gach iarracht an claíomh draíochta a tharraingt amach. Theip orthu go léir. Bhí an claíomh gafa, greamaithe sa charraig. Ach nuair a chuir Artúr a lámh ar an gclaíomh, shleamhnaigh sé amach go deas réidh. Ní raibh muintir na tíre sásta glacadh leis mar rí i dtosach. De réir a chéile, áfach, thuig siad go raibh sé stuama, cineálta agus thug siad an-ghrá dó. Le cabhair ó Merlin, thug Artúr an tír slán ó ionróirí. Nuair a bhí an troid thart, chuir Artúr an claíomh ar ais sa charraig. Ag

('https://www.leighleat.com/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBclFuIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--a6f03c6d6a35f802262cd6a4a26c3108ef6e8449/Cli%CC%81ona%20&%20an%20Clai%CC%81omh%20Cb%201%20lch%201.mp3',
 'Caibidil 1: An Rí Artúr\xa0\n\n\nFadó fadó bhí laoch óg misniúil ann darbh ainm Artúr. Theastaigh uaidh a thír a thabhairt slán ó ionradh agus ó ansmacht. Chabhraigh draoi darbh ainm Merlin leis. Rinne Merlin claíomh draíochta a bhí greamaithe go docht i gcarraig mhór. Seo mar a bhí scríofa ar an gcarraig: Pé duine atá ábalta an claíomh seo a tharraingt as an gcarraig, beidh an duine sin ina cheannaire dlistineach ar an tír. \nRinne na fir ba láidre sa tír gach iarracht an claíomh draíochta a tharraingt amach. Theip orthu go léir. Bhí an claíomh gafa, greamaithe sa charraig. Ach nuair a chuir Artúr a lámh ar an gclaíomh, shleamhnaigh sé amach go deas réidh. Ní raibh muintir na tíre sásta glacadh leis mar rí i dtosach. De réir a chéile, áf

In [26]:
test_audio = "https://www.leighleat.com/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBclFuIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--a6f03c6d6a35f802262cd6a4a26c3108ef6e8449/Cli%CC%81ona%20&%20an%20Clai%CC%81omh%20Cb%201%20lch%201.mp3"

In [27]:
aud = requests.get(test_audio)